In [ ]:
# Reshaping and pivot tables
# pandas provides methods for manipulating a Series and DataFrame to alter the representation of the data for further data processing or data summarization.

# pivot() and pivot_table(): Group unique values within one or more discrete categories.

# stack() and unstack(): Pivot a column or row level to the opposite axis respectively.

# melt() and wide_to_long(): Unpivot a wide DataFrame to a long format.

# get_dummies() and from_dummies(): Conversions with indicator variables.

# explode(): Convert a column of list-like values to individual rows.

# crosstab(): Calculate a cross-tabulation of multiple 1 dimensional factor arrays.

# cut(): Transform continuous variables to discrete, categorical values

# factorize(): Encode 1 dimensional variables into integer labels.

In [ ]:
# pivot()
# Data is often stored in so-called “stacked” or “record” format. In a “record” or “wide” format, typically there is one row for each subject. In the “stacked” or “long” format there are multiple rows for each subject where applicable.

In [ ]:
data = {
   "value": range(12),
   "variable": ["A"] * 3 + ["B"] * 3 + ["C"] * 3 + ["D"] * 3,
   "date": pd.to_datetime(["2020-01-03", "2020-01-04", "2020-01-05"] * 4)
}


df = pd.DataFrame(data)

In [ ]:
# To perform time series operations with each unique variable, a better representation would be where the columns are the unique variables and an index of dates identifies individual observations. To reshape the data into this form, we use the DataFrame.pivot() method (also implemented as a top level function pivot()):

In [ ]:
pivoted = df.pivot(index="date", columns="variable", values="value")

pivoted


In [ ]:
# If the values argument is omitted, and the input DataFrame has more than one column of values which are not used as column or index inputs to pivot(), then the resulting “pivoted” DataFrame will have hierarchical columns whose topmost level indicates the respective value column:

In [ ]:
df["value2"] = df["value"] * 2

pivoted = df.pivot(index="date", columns="variable")

pivoted

In [ ]:
# pivot_table()
# While pivot() provides general purpose pivoting with various data types, pandas also provides pivot_table() or pivot_table() for pivoting with aggregation of numeric data.

# The function pivot_table() can be used to create spreadsheet-style pivot tables. See the cookbook for some advanced strategies.

In [1]:
import datetime
import pandas as pd
import numpy as np

In [3]:
df = pd.DataFrame(
    {
        "A": ["one", "one", "two", "three"] * 6,
        "B": ["A", "B", "C"] * 8,
        "C": ["foo", "foo", "foo", "bar", "bar", "bar"] * 4,
        "D": np.random.randn(24),
        "E": np.random.randn(24),
        "F": [datetime.datetime(2013, i, 1) for i in range(1, 13)]
        + [datetime.datetime(2013, i, 15) for i in range(1, 13)],
    }
)

In [4]:
pd.pivot_table(df, values="D", index=["A", "B"], columns=["C"])

C             bar       foo
A     B                    
one   A  1.617126  0.899944
      B  0.321783 -1.462782
      C -1.008149 -0.042811
three A  0.516792       NaN
      B       NaN  0.934805
      C  0.316769       NaN
two   A       NaN -0.157541
      B -0.482353       NaN
      C       NaN  0.419695

In [5]:
pd.pivot_table(
    df, values=["D", "E"],
    index=["B"],
    columns=["A", "C"],
    aggfunc="sum",
)

D                                                          E  \
A       one               three                two                 one   
C       bar       foo       bar      foo       bar       foo       bar   
B                                                                        
A  3.234251  1.799887  1.033584      NaN       NaN -0.315081  1.234517   
B  0.643567 -2.925563       NaN  1.86961 -0.964705       NaN -3.284723   
C -2.016298 -0.085622  0.633538      NaN       NaN  0.839390  0.379244   

                                                     
A               three                 two            
C       foo       bar       foo       bar       foo  
B                                                    
A -0.330023 -1.864289       NaN       NaN -1.790207  
B  0.336901       NaN -0.742654  1.423737       NaN  
C  1.507824 -0.382420       NaN       NaN -0.302323

In [6]:
pd.pivot_table(
    df, values="E",
    index=["B", "C"],
    columns=["A"],
    aggfunc=["sum", "mean"],
)

sum                          mean                    
A           one     three       two       one     three       two
B C                                                              
A bar  1.234517 -1.864289       NaN  0.617259 -0.932144       NaN
  foo -0.330023       NaN -1.790207 -0.165012       NaN -0.895104
B bar -3.284723       NaN  1.423737 -1.642362       NaN  0.711869
  foo  0.336901 -0.742654       NaN  0.168450 -0.371327       NaN
C bar  0.379244 -0.382420       NaN  0.189622 -0.191210       NaN
  foo  1.507824       NaN -0.302323  0.753912       NaN -0.151161

In [ ]:
# The result is a DataFrame potentially having a MultiIndex on the index or column. If the values column name is not given, the pivot table will include all of the data in an additional level of hierarchy in the columns:

In [7]:
pd.pivot_table(df[["A", "B", "C", "D", "E"]], index=["A", "B"], columns=["C"])

D                   E          
C             bar       foo       bar       foo
A     B                                        
one   A  1.617126  0.899944  0.617259 -0.165012
      B  0.321783 -1.462782 -1.642362  0.168450
      C -1.008149 -0.042811  0.189622  0.753912
three A  0.516792       NaN -0.932144       NaN
      B       NaN  0.934805       NaN -0.371327
      C  0.316769       NaN -0.191210       NaN
two   A       NaN -0.157541       NaN -0.895104
      B -0.482353       NaN  0.711869       NaN
      C       NaN  0.419695       NaN -0.151161

In [9]:
pd.pivot_table(df, values="D", index=pd.Grouper(freq="ME", key="F"), columns="C")
# Out[16]: 

C,bar,foo
F,,
2013-01-31,NaN,0.899944
2013-02-28,NaN,-1.462782
2013-03-31,NaN,0.419695
2013-04-30,0.516792,NaN
2013-05-31,0.321783,NaN
2013-06-30,-1.008149,NaN
2013-07-31,NaN,-0.157541
2013-08-31,NaN,0.934805
2013-09-30,NaN,-0.042811


In [ ]:
# Adding margins
# Passing margins=True to pivot_table() will add a row and column with an All label with partial group aggregates across the categories on the rows and columns:

In [10]:
table = df.pivot_table(
    index=["A", "B"],
    columns="C",
    values=["D", "E"],
    margins=True,
    aggfunc="std"
)
table

D                             E                    
C             bar       foo       All       bar       foo       All
A     B                                                            
one   A  1.234793  1.645143  1.257718  0.401499  0.052975  0.508578
      B  1.171739  0.038642  1.232766  1.535276  0.582352  1.411293
      C  1.757436  0.502770  1.193487  0.322636  0.079214  0.378062
three A  1.458486       NaN  1.458486  0.401844       NaN  0.401844
      B       NaN  0.200215  0.200215       NaN  1.496437  1.496437
      C  2.043500       NaN  2.043500  1.317818       NaN  1.317818
two   A       NaN  0.799145  0.799145       NaN  0.658534  0.658534
      B  0.215517       NaN  0.215517  0.256798       NaN  0.256798
      C       NaN  0.022493  0.022493       NaN  0.459866  0.459866
All      1.363756  1.025848  1.181628  1.092170  0.754335  0.919305

In [ ]:
# Additionally, you can call DataFrame.stack() to display a pivoted DataFrame as having a multi-level index:

In [11]:
table.stack(future_stack=True)

D         E
A     B C                      
one   A bar  1.234793  0.401499
        foo  1.645143  0.052975
        All  1.257718  0.508578
      B bar  1.171739  1.535276
        foo  0.038642  0.582352
        All  1.232766  1.411293
      C bar  1.757436  0.322636
        foo  0.502770  0.079214
        All  1.193487  0.378062
three A bar  1.458486  0.401844
        foo       NaN       NaN
        All  1.458486  0.401844
      B bar       NaN       NaN
        foo  0.200215  1.496437
        All  0.200215  1.496437
      C bar  2.043500  1.317818
        foo       NaN       NaN
        All  2.043500  1.317818
two   A bar       NaN       NaN
        foo  0.799145  0.658534
        All  0.799145  0.658534
      B bar  0.215517  0.256798
        foo       NaN       NaN
        All  0.215517  0.256798
      C bar       NaN       NaN
        foo  0.022493  0.459866
        All  0.022493  0.459866
All     bar  1.363756  1.092170
        foo  1.025848  0.754335
        All  1.181628  0.919305

In [ ]:
# stack() and unstack()

In [ ]:
# Closely related to the pivot() method are the related stack() and unstack() methods available on Series and DataFrame. These methods are designed to work together with MultiIndex objects (see the section on hierarchical indexing).

# stack(): “pivot” a level of the (possibly hierarchical) column labels, returning a DataFrame with an index with a new inner-most level of row labels.

# unstack(): (inverse operation of stack()) “pivot” a level of the (possibly hierarchical) row index to the column axis, producing a reshaped DataFrame with a new inner-most level of column labels.

In [12]:
tuples = [
   ["bar", "bar", "baz", "baz", "foo", "foo", "qux", "qux"],
   ["one", "two", "one", "two", "one", "two", "one", "two"],
]


index = pd.MultiIndex.from_arrays(tuples, names=["first", "second"])

df = pd.DataFrame(np.random.randn(8, 2), index=index, columns=["A", "B"])

df2 = df[:4]

df2

A         B
first second                    
bar   one     0.407434  0.909650
      two    -1.849410 -0.288512
baz   one     0.378830  0.419210
      two     1.215430 -0.060139

In [ ]:
# The stack() function “compresses” a level in the DataFrame columns to produce either:

# A Series, in the case of a Index in the columns.

# A DataFrame, in the case of a MultiIndex in the columns.

# If the columns have a MultiIndex, you can choose which level to stack. The stacked level becomes the new lowest level in a MultiIndex on the columns:

In [13]:
stacked = df2.stack(future_stack=True)

stacked

first  second   
bar    one     A    0.407434
               B    0.909650
       two     A   -1.849410
               B   -0.288512
baz    one     A    0.378830
               B    0.419210
       two     A    1.215430
               B   -0.060139
dtype: float64

In [ ]:
# With a “stacked” DataFrame or Series (having a MultiIndex as the index), the inverse operation of stack() is unstack(), which by default unstacks the last level:

In [14]:
stacked.unstack()

A         B
first second                    
bar   one     0.407434  0.909650
      two    -1.849410 -0.288512
baz   one     0.378830  0.419210
      two     1.215430 -0.060139

In [15]:
stacked.unstack(2)

A         B
first second                    
bar   one     0.407434  0.909650
      two    -1.849410 -0.288512
baz   one     0.378830  0.419210
      two     1.215430 -0.060139

In [ ]:
# If the indexes have names, you can use the level names instead of specifying the level numbers:

In [16]:
stacked.unstack("second")

second        one       two
first                      
bar   A  0.407434 -1.849410
      B  0.909650 -0.288512
baz   A  0.378830  1.215430
      B  0.419210 -0.060139

In [ ]:
# Notice that the stack() and unstack() methods implicitly sort the index levels involved. Hence a call to stack() and then unstack(), or vice versa, will result in a sorted copy of the original DataFrame or Series:

In [17]:
index = pd.MultiIndex.from_product([[2, 1], ["a", "b"]])

df = pd.DataFrame(np.random.randn(4), index=index, columns=["A"])

df

A
2 a  0.203269
  b -1.096269
1 a  0.458733
  b -0.613080

In [ ]:
# Multiple levels
# You may also stack or unstack more than one level at a time by passing a list of levels, in which case the end result is as if each level in the list were processed individually.



In [36]:
columns = pd.MultiIndex.from_tuples(
    [
        ("A", "cat", "long"),
        ("B", "cat", "long"),
        ("A", "dog", "short"),
        ("B", "dog", "short"),
    ],
    names=["exp", "animal", "hair_length"],
)


df = pd.DataFrame(np.random.randn(4, 4), columns=columns)


In [37]:
df.stack(level=["animal", "hair_length"], future_stack=True)

exp                          A         B
  animal hair_length                    
0 cat    long        -0.647321 -1.099500
  dog    short       -0.027107 -1.477075
1 cat    long         1.283849  1.035905
  dog    short        1.278211  0.884326
2 cat    long        -0.223978 -1.261861
  dog    short       -1.222198 -1.269374
3 cat    long        -0.358049  1.525807
  dog    short       -0.494875  0.332488

In [ ]:
# The list of levels can contain either level names or level numbers but not a mixture of the two.

In [38]:
df.stack(level=[1, 2], future_stack=True)

exp                          A         B
  animal hair_length                    
0 cat    long        -0.647321 -1.099500
  dog    short       -0.027107 -1.477075
1 cat    long         1.283849  1.035905
  dog    short        1.278211  0.884326
2 cat    long        -0.223978 -1.261861
  dog    short       -1.222198 -1.269374
3 cat    long        -0.358049  1.525807
  dog    short       -0.494875  0.332488

In [ ]:
# Missing data
# Unstacking can result in missing values if subgroups do not have the same set of labels. By default, missing values will be replaced with the default fill value for that data type.

In [39]:
columns = pd.MultiIndex.from_tuples(
    [
        ("A", "cat"),
        ("B", "dog"),
        ("B", "cat"),
        ("A", "dog"),
    ],
    names=["exp", "animal"],
)

index = pd.MultiIndex.from_product(
    [("bar", "baz", "foo", "qux"), ("one", "two")], names=["first", "second"]
)


df = pd.DataFrame(np.random.randn(8, 4), index=index, columns=columns)

df3 = df.iloc[[0, 1, 4, 7], [1, 2]]

df3

exp                  B          
animal             dog       cat
first second                    
bar   one     0.239510  0.099623
      two    -0.292651  1.161758
foo   one     1.379979  0.075584
qux   two    -0.247370  1.689457

In [40]:
df3.unstack()

exp            B                              
animal       dog                 cat          
second       one       two       one       two
first                                         
bar     0.239510 -0.292651  0.099623  1.161758
foo     1.379979       NaN  0.075584       NaN
qux          NaN -0.247370       NaN  1.689457

In [41]:
# The missing value can be filled with a specific value with the fill_value argument.

df3.unstack(fill_value=-1e9)

exp                B                                          
animal           dog                         cat              
second           one           two           one           two
first                                                         
bar     2.395098e-01 -2.926513e-01  9.962320e-02  1.161758e+00
foo     1.379979e+00 -1.000000e+09  7.558443e-02 -1.000000e+09
qux    -1.000000e+09 -2.473703e-01 -1.000000e+09  1.689457e+00

In [ ]:
# melt() and wide_to_long()

In [ ]:
# The top-level melt() function and the corresponding DataFrame.melt() are useful to massage a DataFrame into a format where one or more columns are identifier variables, while all other columns, considered measured variables, are “unpivoted” to the row axis, leaving just two non-identifier columns, “variable” and “value”. The names of those columns can be customized by supplying the var_name and value_name parameters.

In [42]:
cheese = pd.DataFrame(
    {
        "first": ["John", "Mary"],
        "last": ["Doe", "Bo"],
        "height": [5.5, 6.0],
        "weight": [130, 150],
    }
)


cheese

,first,last,height,weight
0,John,Doe,5.5,130
1,Mary,Bo,6.0,150


In [43]:

cheese.melt(id_vars=["first", "last"])

,first,last,variable,value
0,John,Doe,height,5.5
1,Mary,Bo,height,6.0
2,John,Doe,weight,130.0
3,Mary,Bo,weight,150.0


In [44]:
cheese.melt(id_vars=["first", "last"], var_name="quantity")

,first,last,quantity,value
0,John,Doe,height,5.5
1,Mary,Bo,height,6.0
2,John,Doe,weight,130.0
3,Mary,Bo,weight,150.0


In [45]:
index = pd.MultiIndex.from_tuples([("person", "A"), ("person", "B")])

cheese = pd.DataFrame(
    {
        "first": ["John", "Mary"],
        "last": ["Doe", "Bo"],
        "height": [5.5, 6.0],
        "weight": [130, 150],
    },
    index=index,
)


cheese

first last  height  weight
person A  John  Doe     5.5     130
       B  Mary   Bo     6.0     150

In [46]:
cheese.melt(id_vars=["first", "last"])

,first,last,variable,value
0,John,Doe,height,5.5
1,Mary,Bo,height,6.0
2,John,Doe,weight,130.0
3,Mary,Bo,weight,150.0


In [47]:
cheese.melt(id_vars=["first", "last"], ignore_index=False)

first last variable  value
person A  John  Doe   height    5.5
       B  Mary   Bo   height    6.0
       A  John  Doe   weight  130.0
       B  Mary   Bo   weight  150.0

In [ ]:
# wide_to_long() is similar to melt() with more customization for column matching.

In [48]:
dft = pd.DataFrame(
    {
        "A1970": {0: "a", 1: "b", 2: "c"},
        "A1980": {0: "d", 1: "e", 2: "f"},
        "B1970": {0: 2.5, 1: 1.2, 2: 0.7},
        "B1980": {0: 3.2, 1: 1.3, 2: 0.1},
        "X": dict(zip(range(3), np.random.randn(3))),
    }
)


dft["id"] = dft.index

In [49]:
pd.wide_to_long(dft, ["A", "B"], i="id", j="year")

,,X,A,B
id,year,,,
0,1970,-0.577542,a,2.5
1,1970,-0.973598,b,1.2
2,1970,-1.228564,c,0.7
0,1980,-0.577542,d,3.2
1,1980,-0.973598,e,1.3
2,1980,-1.228564,f,0.1


In [ ]:
# get_dummies() and from_dummies()
# To convert categorical variables of a Series into a “dummy” or “indicator”, get_dummies() creates a new DataFrame with columns of the unique variables and the values representing the presence of those variables per row.

In [50]:
df = pd.DataFrame({"key": list("bbacab"), "data1": range(6)})

pd.get_dummies(df["key"])

,a,b,c
0,False,True,False
1,False,True,False
2,True,False,False
3,False,False,True
4,True,False,False
5,False,True,False


In [51]:
df["key"].str.get_dummies()

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [ ]:
# prefix adds a prefix to the the column names which is useful for merging the result with the original DataFrame:

In [52]:
dummies = pd.get_dummies(df["key"], prefix="key")

dummies

,key_a,key_b,key_c
0,False,True,False
1,False,True,False
2,True,False,False
3,False,False,True
4,True,False,False
5,False,True,False


In [53]:
df[["data1"]].join(dummies)

,data1,key_a,key_b,key_c
0,0,False,True,False
1,1,False,True,False
2,2,True,False,False
3,3,False,False,True
4,4,True,False,False
5,5,False,True,False


In [ ]:
# This function is often used along with discretization functions like cut():

In [55]:
values = np.random.randn(10)
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,False,False,False,False,False
1,False,False,False,True,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
5,False,False,False,False,False
6,False,True,False,False,False
7,False,False,False,False,False
8,False,False,True,False,False
9,False,False,False,True,False


In [ ]:
# get_dummies() also accepts a DataFrame. By default, object, string, or categorical type columns are encoded as dummy variables with other columns unaltered.

In [57]:
df = pd.DataFrame({"A": ["a", "b", "a"], "B": ["c", "c", "b"], "C": [1, 2, 3]})

pd.get_dummies(df)

,C,A_a,A_b,B_b,B_c
0,1,True,False,False,True
1,2,False,True,False,True
2,3,True,False,True,False


In [ ]:
# Specifying the columns keyword will encode a column of any type.

In [58]:
pd.get_dummies(df, columns=["A"])

,B,C,A_a,A_b
0,c,1,True,False
1,c,2,False,True
2,b,3,True,False


In [ ]:
# As with the Series version, you can pass values for the prefix and prefix_sep. By default the column name is used as the prefix and _ as the prefix separator. You can specify prefix and prefix_sep in 3 ways:

# string: Use the same value for prefix or prefix_sep for each column to be encoded.

# list: Must be the same length as the number of columns being encoded.

# dict: Mapping column name to prefix.

In [59]:
simple = pd.get_dummies(df, prefix="new_prefix")

simple

,C,new_prefix_a,new_prefix_b,new_prefix_b,new_prefix_c
0,1,True,False,False,True
1,2,False,True,False,True
2,3,True,False,True,False


In [60]:
from_list = pd.get_dummies(df, prefix=["from_A", "from_B"])

from_list

,C,from_A_a,from_A_b,from_B_b,from_B_c
0,1,True,False,False,True
1,2,False,True,False,True
2,3,True,False,True,False


In [ ]:
# To avoid collinearity when feeding the result to statistical models, specify drop_first=True.

In [61]:
s = pd.Series(list("abcaa"))

pd.get_dummies(s)

,a,b,c
0,True,False,False
1,False,True,False
2,False,False,True
3,True,False,False
4,True,False,False


In [62]:

pd.get_dummies(s, drop_first=True)

,b,c
0,False,False
1,True,False
2,False,True
3,False,False
4,False,False


In [ ]:
# When a column contains only one level, it will be omitted in the result.

In [63]:
df = pd.DataFrame({"A": list("aaaaa"), "B": list("ababc")})

pd.get_dummies(df)

,A_a,B_a,B_b,B_c
0,True,True,False,False
1,True,False,True,False
2,True,True,False,False
3,True,False,True,False
4,True,False,False,True


In [ ]:
# The values can be cast to a different type using the dtype argument.

In [64]:
df = pd.DataFrame({"A": list("abc"), "B": [1.1, 2.2, 3.3]})

pd.get_dummies(df, dtype=np.float32).dtypes

B      float64
A_a    float32
A_b    float32
A_c    float32
dtype: object

In [ ]:
# from_dummies() converts the output of get_dummies() back into a Series of categorical values from indicator values.

In [65]:
df = pd.DataFrame({"prefix_a": [0, 1, 0], "prefix_b": [1, 0, 1]})

df

pd.from_dummies(df, sep="_")

,prefix
0,b
1,a
2,b


In [ ]:
# Dummy coded data only requires k - 1 categories to be included, in this case the last category is the default category. The default category can be modified with default_category.

In [66]:
df = pd.DataFrame({"prefix_a": [0, 1, 0]})

In [69]:
df = pd.DataFrame({"prefix_a": [0, 1, 0]})
df

,prefix_a
0,0
1,1
2,0


In [70]:
pd.from_dummies(df, sep="_", default_category="b")

,prefix
0,b
1,a
2,b


In [ ]:
# When transforming a DataFrame using melt(), the index will be ignored. The original index values can be kept by setting the ignore_index=False parameter to False (default is True). ignore_index=False will however duplicate index values

In [ ]:
# explode()

In [ ]:
# For a DataFrame column with nested, list-like values, explode() will transform each list-like value to a separate row. The resulting Index will be duplicated corresponding to the index label from the original row:

In [18]:
keys = ["panda1", "panda2", "panda3"]

values = [["eats", "shoots"], ["shoots", "leaves"], ["eats", "leaves"]]

df = pd.DataFrame({"keys": keys, "values": values})
df

,keys,values
0,panda1,"[eats, shoots]"
1,panda2,"[shoots, leaves]"
2,panda3,"[eats, leaves]"


In [19]:
df["values"].explode()

0      eats
0    shoots
1    shoots
1    leaves
2      eats
2    leaves
Name: values, dtype: object

In [ ]:
# DataFrame.explode can also explode the column in the DataFrame.

In [20]:
df.explode("values")

,keys,values
0,panda1,eats
0,panda1,shoots
1,panda2,shoots
1,panda2,leaves
2,panda3,eats
2,panda3,leaves


In [ ]:
# Series.explode() will replace empty lists with a missing value indicator and preserve scalar entries.

In [21]:
s = pd.Series([[1, 2, 3], "foo", [], ["a", "b"]])

s

0    [1, 2, 3]
1          foo
2           []
3       [a, b]
dtype: object

In [22]:
s.explode()

0      1
0      2
0      3
1    foo
2    NaN
3      a
3      b
dtype: object

In [ ]:
# A comma-separated string value can be split into individual values in a list and then exploded to a new row.

In [23]:
df = pd.DataFrame([{"var1": "a,b,c", "var2": 1}, {"var1": "d,e,f", "var2": 2}])

df.assign(var1=df.var1.str.split(",")).explode("var1")

,var1,var2
0,a,1
0,b,1
0,c,1
1,d,2
1,e,2
1,f,2


In [ ]:
# crosstab()
# Use crosstab() to compute a cross-tabulation of two (or more) factors. By default crosstab() computes a frequency table of the factors unless an array of values and an aggregation function are passed.

# Any Series passed will have their name attributes used unless row or column names for the cross-tabulation are specified

In [24]:
a = np.array(["foo", "foo", "bar", "bar", "foo", "foo"], dtype=object)

b = np.array(["one", "one", "two", "one", "two", "one"], dtype=object)

c = np.array(["dull", "dull", "shiny", "dull", "dull", "shiny"], dtype=object)

pd.crosstab(a, [b, c], rownames=["a"], colnames=["b", "c"])

b    one        two      
c   dull shiny dull shiny
a                        
bar    1     0    0     1
foo    2     1    1     0

In [ ]:
# If crosstab() receives only two Series, it will provide a frequency table.

In [25]:
df = pd.DataFrame(
    {"A": [1, 2, 2, 2, 2], "B": [3, 3, 4, 4, 4], "C": [1, 1, np.nan, 1, 1]}
)


df

,A,B,C
0,1,3,1.0
1,2,3,1.0
2,2,4,NaN
3,2,4,1.0
4,2,4,1.0


In [26]:
pd.crosstab(df["A"], df["B"])

B,3,4
A,,
1,1,0
2,1,3


In [28]:
foo = pd.Categorical(["a", "b"], categories=["a", "b", "c"])

bar = pd.Categorical(["d", "e"], categories=["d", "e", "f"])

pd.crosstab(foo, bar)

col_0,d,e
row_0,,
a,1,0
b,0,1


In [ ]:
# For Categorical data, to include all of data categories even if the actual data does not contain any instances of a particular category, use dropna=False.

In [29]:
pd.crosstab(foo, bar, dropna=False)

col_0,d,e,f
row_0,,,
a,1,0,0
b,0,1,0
c,0,0,0


In [31]:
# Normalization
# Frequency tables can also be normalized to show percentages rather than counts using the normalize argument:

In [32]:
pd.crosstab(df["A"], df["B"], normalize=True)

B,3,4
A,,
1,0.2,0.0
2,0.2,0.6


In [ ]:
# normalize can also normalize values within each row or within each column:

In [33]:
pd.crosstab(df["A"], df["B"], normalize="columns")

B,3,4
A,,
1,0.5,0.0
2,0.5,1.0


In [ ]:
# crosstab() can also accept a third Series and an aggregation function (aggfunc) that will be applied to the values of the third Series within each group defined by the first two Series:

In [34]:
pd.crosstab(df["A"], df["B"], values=df["C"], aggfunc="sum")

B,3,4
A,,
1,1.0,NaN
2,1.0,2.0


In [ ]:
# Adding margins

In [ ]:
# margins=True will add a row and column with an All label with partial group aggregates across the categories on the rows and columns:

In [35]:
pd.crosstab(
    df["A"], df["B"], values=df["C"], aggfunc="sum", normalize=True, margins=True
)

B,3,4,All
A,,,
1,0.25,0.0,0.25
2,0.25,0.5,0.75
All,0.50,0.5,1.00
